### 1. Setup inicial do Spark e Postgres

In [1]:
import sys
sys.path.append("../libs")
from utils import silenciar_warnings, pgconfig_init, pg_conn, pg_executar_sql, spark_init, spark_read_jdbc

# Silenciar warnings/logs
silenciar_warnings()

#### 1.1 Spark

In [2]:
# Inicializar Spark
spark = spark_init("IBGE_Gold")

:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /Users/hgirardi/.ivy2.5.2/cache
The jars for the packages stored in: /Users/hgirardi/.ivy2.5.2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8bf2c441-7074-476e-9ec0-24e83baf5486;1.0
	confs: [default]
	found org.postgresql#postgresql;42.7.3 in central
	found org.checkerframework#checker-qual;3.42.0 in central
:: resolution report :: resolve 49ms :: artifacts dl 2ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------

✅ Spark inicializado: IBGE_Gold [local[*]]


#### 1.2 Postgres

In [3]:
# initizar configuração do PostgreSQL
pgConfig = pgconfig_init()

# criar schema silver no PostgreSQL, se não existir
with pg_conn(pgConfig) as conn:
    pg_executar_sql(conn, "CREATE SCHEMA IF NOT EXISTS gold;")

✅ Configuração do Postgres carregada.
== Executando SQL no Postgres ==
CREATE SCHEMA IF NOT EXISTS gold;
✅ SQL executado!
== Processo finalizado ==



### 2. Carregar Silver

In [4]:
# carregar tabela silver.covid
df_silver = spark_read_jdbc(spark, "silver.covid", pgConfig)

df_silver.createOrReplaceTempView("silver_covid")
spark.sql("CACHE TABLE silver_covid")  # acelera as próximas queries
_ = spark.table("silver_covid").limit(1).count()

In [5]:
df_silver.show(20, truncate=False)
df_silver.printSchema()

+-----------------+----------+------------+------------------+-----+------+--------+------------------------------------+---------------+----------+--------------+-----------------+---------------------+------------------+----------------------+--------------------------------------------------------------------------------------------------------------------------------+-----+-----+------------+--------------------+----------+---------+------+--------------+------+---------+--------------------+------------+--------+---------------+----------------+-------------------+--------------------+--------------+
|estado           |semana_mes|mes_pesquisa|situacao_domicilio|idade|sexo  |cor_raca|escolaridade                        |fez_teste_covid|teste_swab|resultado_swab|teste_sangue_dedo|resultado_sangue_dedo|teste_sangue_braco|resultado_sangue_braco|restricao_contato                                                                                                               |febre|tosse|d

### 3. Criação do Gold

#### 3.6 mv_positividade_semanal
Materilized View contendo os seguintes dados de todo pais acumulado por semana:
- Número de entrevistas realizadas (respondentes)
- Número de testados (testados)
- Taxa de testados (taxa_testados)
- Número de casos positivos (positivos)
- Taxa de positividade (taxa_positividade)

In [ ]:
with pg_conn(pgConfig) as conn:
    pg_executar_sql(conn, """
        
            CREATE SCHEMA IF NOT EXISTS gold;

            DROP TABLE IF EXISTS gold.covid_base;
            CREATE TABLE gold.covid_base AS
            SELECT
            
                to_char(make_date(2000, mes_pesquisa, 1), 'TMMonth') AS mes,
                mes_pesquisa as mes_numero,
                semana_mes,

                -- chave de agrupamento principal
                estado AS uf,

                -- flags principais
                (fez_teste_covid IS TRUE)::int      AS fl_testou,
                (teste_covid_positivo IS TRUE)::int AS fl_pos,
                (possui_sintoma IS TRUE)::int       AS fl_sint,

                -- domicílio
                (situacao_domicilio = 'Rural')::int  AS fl_rural,
                (situacao_domicilio = 'Urbana')::int AS fl_urbano,

                -- sexo
                (sexo = 'Homem')::int  AS fl_homem,
                (sexo = 'Mulher')::int AS fl_mulher,

                -- cor/raça
                (cor_raca = 'Amarela')::int  AS fl_cor_amarela,
                (cor_raca = 'Branca')::int   AS fl_cor_branca,
                (cor_raca = 'Ignorado')::int AS fl_cor_ignorado,
                (cor_raca = 'Indigena')::int AS fl_cor_indigena,
                (cor_raca = 'Parda')::int    AS fl_cor_parda,
                (cor_raca = 'Preta')::int    AS fl_cor_preta,

                -- escolaridade
                (escolaridade = 'Fundamental completa')::int                 AS fl_esc_fund_comp,
                (escolaridade = 'Fundamental incompleto')::int               AS fl_esc_fund_incomp,
                (escolaridade = 'Médio completo')::int                       AS fl_esc_medio_comp,
                (escolaridade = 'Médio incompleto')::int                     AS fl_esc_medio_incomp,
                (escolaridade = 'Pós-graduação, mestrado ou doutorado')::int AS fl_esc_pos,
                (escolaridade = 'Sem instrução')::int                        AS fl_esc_sem_instrucao,
                (escolaridade = 'Superior completo')::int                    AS fl_esc_sup_comp,
                (escolaridade = 'Superior incompleto')::int                  AS fl_esc_sup_incomp,

                -- idade por década
                (idade BETWEEN  0 AND  9)::int AS fl_id_00_09,
                (idade BETWEEN 10 AND 19)::int AS fl_id_10_19,
                (idade BETWEEN 20 AND 29)::int AS fl_id_20_29,
                (idade BETWEEN 30 AND 39)::int AS fl_id_30_39,
                (idade BETWEEN 40 AND 49)::int AS fl_id_40_49,
                (idade BETWEEN 50 AND 59)::int AS fl_id_50_59,
                (idade BETWEEN 60 AND 69)::int AS fl_id_60_69,
                (idade BETWEEN 70 AND 79)::int AS fl_id_70_79,
                (idade >= 80)::int             AS fl_id_80p
            FROM silver.covid;

            -- 1) Base rotulada (agora incluindo mes/semana)
            CREATE OR REPLACE VIEW gold.v_covid_dim_base AS
            SELECT
                b.uf,
                b.mes,
                b.mes_numero,
                b.semana_mes,
                b.fl_testou, b.fl_pos, b.fl_sint,

                CASE WHEN b.fl_rural=1  THEN 'Rural'
                    WHEN b.fl_urbano=1 THEN 'Urbana' END AS domicilio,

                CASE WHEN b.fl_homem=1  THEN 'Homem'
                    WHEN b.fl_mulher=1 THEN 'Mulher' END AS sexo,

                CASE WHEN b.fl_cor_amarela=1  THEN 'Amarela'
                    WHEN b.fl_cor_branca=1   THEN 'Branca'
                    WHEN b.fl_cor_ignorado=1 THEN 'Ignorado'
                    WHEN b.fl_cor_indigena=1 THEN 'Indigena'
                    WHEN b.fl_cor_parda=1    THEN 'Parda'
                    WHEN b.fl_cor_preta=1    THEN 'Preta' END AS raca,

                CASE WHEN b.fl_esc_fund_comp=1     THEN 'Fundamental completa'
                    WHEN b.fl_esc_fund_incomp=1   THEN 'Fundamental incompleto'
                    WHEN b.fl_esc_medio_comp=1    THEN 'Médio completo'
                    WHEN b.fl_esc_medio_incomp=1  THEN 'Médio incompleto'
                    WHEN b.fl_esc_pos=1           THEN 'Pós-graduação, mestrado ou doutorado'
                    WHEN b.fl_esc_sem_instrucao=1 THEN 'Sem instrução'
                    WHEN b.fl_esc_sup_comp=1      THEN 'Superior completo'
                    WHEN b.fl_esc_sup_incomp=1    THEN 'Superior incompleto' END AS escolaridade,

                CASE WHEN b.fl_id_00_09=1 THEN '00-09'
                    WHEN b.fl_id_10_19=1 THEN '10-19'
                    WHEN b.fl_id_20_29=1 THEN '20-29'
                    WHEN b.fl_id_30_39=1 THEN '30-39'
                    WHEN b.fl_id_40_49=1 THEN '40-49'
                    WHEN b.fl_id_50_59=1 THEN '50-59'
                    WHEN b.fl_id_60_69=1 THEN '60-69'
                    WHEN b.fl_id_70_79=1 THEN '70-79'
                    WHEN b.fl_id_80p=1   THEN '80+' END AS idade_bucket
            FROM gold.covid_base b;

            -- 2) Totais por UF + período 
            CREATE OR REPLACE VIEW gold.v_covid_totais_uf_periodo AS
            SELECT
                uf,
                mes,
                mes_numero,
                semana_mes   AS semana,
                COUNT(*)       AS respondentes,
                SUM(fl_testou) AS testados,
                SUM(fl_sint)   AS possui_sintomas,
                SUM(fl_pos)    AS positivos
            FROM gold.v_covid_dim_base
            GROUP BY uf, mes, mes_numero, semana_mes;

            -- 3) Criar a tabela de pares
            DROP TABLE IF EXISTS gold.covid_agg_pairs;
            CREATE TABLE gold.covid_agg_pairs AS
            -- domicilio x sexo
            SELECT
                uf, mes, mes_numero, semana_mes AS semana,
                'domicilio' AS dim1, domicilio AS val1,
                'sexo'      AS dim2, sexo      AS val2,
                COUNT(*) AS respondentes,
                SUM(fl_testou) AS testados,
                SUM(fl_pos) AS positivos,
                SUM(fl_sint) AS possui_sintomas
            FROM gold.v_covid_dim_base
            WHERE domicilio IS NOT NULL AND sexo IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, domicilio, sexo

            UNION ALL
            -- domicilio x raca
            SELECT
                uf, mes, mes_numero, semana_mes,
                'domicilio', domicilio, 'raca', raca,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE domicilio IS NOT NULL AND raca IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, domicilio, raca

            UNION ALL
            -- domicilio x escolaridade
            SELECT
                uf, mes, mes_numero, semana_mes,
                'domicilio', domicilio, 'escolaridade', escolaridade,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE domicilio IS NOT NULL AND escolaridade IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, domicilio, escolaridade

            UNION ALL
            -- domicilio x idade
            SELECT
                uf, mes, mes_numero, semana_mes,
                'domicilio', domicilio, 'idade', idade_bucket,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE domicilio IS NOT NULL AND idade_bucket IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, domicilio, idade_bucket

            UNION ALL
            -- sexo x raca
            SELECT
                uf, mes, mes_numero, semana_mes,
                'sexo', sexo, 'raca', raca,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE sexo IS NOT NULL AND raca IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, sexo, raca

            UNION ALL
            -- sexo x escolaridade
            SELECT
                uf, mes, mes_numero, semana_mes,
                'sexo', sexo, 'escolaridade', escolaridade,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE sexo IS NOT NULL AND escolaridade IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, sexo, escolaridade

            UNION ALL
            -- sexo x idade
            SELECT
                uf, mes, mes_numero, semana_mes,
                'sexo', sexo, 'idade', idade_bucket,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE sexo IS NOT NULL AND idade_bucket IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, sexo, idade_bucket

            UNION ALL
            -- raca x escolaridade
            SELECT
                uf, mes, mes_numero, semana_mes,
                'raca', raca, 'escolaridade', escolaridade,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE raca IS NOT NULL AND escolaridade IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, raca, escolaridade

            UNION ALL
            -- raca x idade
            SELECT
                uf, mes, mes_numero, semana_mes,
                'raca', raca, 'idade', idade_bucket,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE raca IS NOT NULL AND idade_bucket IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, raca, idade_bucket

            UNION ALL
            -- escolaridade x idade
            SELECT
                uf, mes, mes_numero, semana_mes,
                'escolaridade', escolaridade, 'idade', idade_bucket,
                COUNT(*), SUM(fl_testou), SUM(fl_pos), SUM(fl_sint)
            FROM gold.v_covid_dim_base
            WHERE escolaridade IS NOT NULL AND idade_bucket IS NOT NULL
            GROUP BY uf, mes, mes_numero, semana_mes, escolaridade, idade_bucket
            ;

            -- índices para filtros
            CREATE INDEX IF NOT EXISTS idx_pairs_periodo_uf ON gold.covid_agg_pairs(uf, mes_numero, semana);
            CREATE INDEX IF NOT EXISTS idx_pairs_dims ON gold.covid_agg_pairs(dim1, val1, dim2, val2);

            -- 4) View com TAXAS (denominadores por UF + período)
            CREATE OR REPLACE VIEW gold.v_covid_agg_pairs_tx AS
            SELECT
                p.*,
                (p.testados::numeric / NULLIF(t.testados, 0))           AS taxa_testados,
                (p.positivos::numeric / NULLIF(t.positivos, 0))         AS taxa_positivos,
                (p.possui_sintomas::numeric / NULLIF(t.possui_sintomas, 0)) AS taxa_sintomas
            FROM gold.covid_agg_pairs p
            JOIN gold.v_covid_totais_uf_periodo t
            ON t.uf = p.uf AND t.mes = p.mes AND t.semana = p.semana;

    """)
    print("✅ Gold criado com sucesso!")


KeyboardInterrupt: 